# Imports

In [1]:
%load_ext autoreload
%autoreload 2
import sys
from nlppen.extraccion.utils.Txt2Numbers import Txt2Numbers
from nlppen.analisis import Analisis
from nlppen.seleccion import Seleccion
from nlppen.spark_udfs import solo_portanto, spark_get_spacy
from nlppen.sentencias_estructurales import SentenciasEstructurales
from pyspark.sql import SparkSession
from pyspark.sql.functions import length

## Initialize spark context

In [2]:
spark = (SparkSession
         .builder
         .appName("Transforming sentences")
         .config("spark.num.executors", "1")
         .config("spark.executor.memory", "12g")
         .config("spark.executor.cores", "2")
         .config("spark.driver.memory", "10g")
         .config("spark.memory.offHeap.enabled", True)
         .config("spark.memory.offHeap.size", "64g")
         .config("spark.sql.execution.arrow.pyspark.enabled", "true")
         .getOrCreate())


sc = spark.sparkContext
sc.addFile('/home/jovyan/Work/ej/paquetes/nlppen/nlppen.zip')
sc.uiWebUrl

21/10/05 21:17:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/10/05 21:17:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/10/05 21:17:06 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


'http://69abb479a5e1:4042'

# Buscar terminos en la sección de por lo tanto de la sentencia y aplicar filtro del dataset

In [3]:
terminos = {
    'seguimiento': [r'\bseguimiento\b'],
    'se ordena': [r'\bse ordena\b'],
    'plan': [r'\bplan\b'],
    'plazo': [r'\bplazo\b']
}


seleccion = Seleccion(terminos, spark, parquet_path='../../src/datasets/complete', datasets_path='./datasets/estructurales')
seleccion.filtrar_sentencias(preprocess=solo_portanto, keepRowEmpty=True)

estructurales = SentenciasEstructurales(seleccion)

21/10/05 21:17:12 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
seleccion.sdf.columns

['archivo',
 'txt',
 'cleanText',
 'num',
 'fechahora_ext',
 'redactor_ext',
 'tipoResolucion_ext',
 'termino_ext',
 'voto_salvado',
 'tema',
 'Recurrente_ents',
 'Cantón_ents',
 'Fecha_ents',
 'Constitución_ents',
 'Redactor_ents',
 'Ley_ents',
 'EntidadPublica_ents',
 'tema_prob',
 'termino_lst',
 'expediente',
 'Fechahora_ents',
 'CitaSentencias_ents',
 'Recurrido_ents',
 'Reglamento_ents',
 'Magistrado_ents',
 '_id',
 'anno',
 'seguimiento',
 'se_ordena',
 'plan',
 'plazo']

# Formar dataset de sentencias estructurales

In [5]:
from pyspark.sql.types import *

columnas = {
    'se ordena PER' : ArrayType(StringType()),
    'se ordena LOC' : ArrayType(StringType()),
    'se ordena ORG' : ArrayType(StringType()),
    'se ordena MISC' : ArrayType(StringType()),
    'se ordena GPE' : ArrayType(StringType()),
    'se ordena Ent Pub' : ArrayType(StringType())
}
estructurales.separarSeOrdena(columnas, True, True)

columnas = {
    'extension sentencia' : IntegerType(),
    'extension por lo tanto' : IntegerType()
}
estructurales.extraerExtension(columnas, True)

columnas = {
    'plazosDefinidos' : ArrayType(TimestampType())
}
estructurales.plazosDefinidos(columnas, True)

columnas = {
    'FechaSolicitud' : TimestampType(),
}

estructurales.extrarFechaRecibido(columnas, True)
#Sobreescribir el dataset de filtro de sentencias con las nuevas columnas
estructurales.seleccion.guardarDatos()

AnalysisException: Found duplicate column(s) when inserting into file:/home/jovyan/Work/ej/paquetes/nlppen/datasets/estructurales/terminos.parquet: `extension_por_lo_tanto`, `extension_sentencia`, `plazosdefinidos`, `se_ordena_ent_pub`, `se_ordena_gpe`, `se_ordena_loc`, `se_ordena_misc`, `se_ordena_org`, `se_ordena_per`

In [6]:
estructurales.seleccion.sdf.limit(15).toPandas()

/opt/conda/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:87: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: ArrayType(TimestampType,true)
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


,archivo,txt,cleanText,num,fechahora_ext,redactor_ext,tipoResolucion_ext,termino_ext,voto_salvado,tema,...,plan,plazo,se_ordena_PER,se_ordena_LOC,se_ordena_ORG,se_ordena_MISC,se_ordena_GPE,extension_sentencia,extension_por_lo_tanto,plazosDefinidos
0,87D50.html,\n*060123060007CO* Exp: 06-012306-0007-CO Res....,co exp co res sal constitucional cort suprem j...,94847,2006-11-29 18:02:00,Vargas Benavides,Recurso de Amparo,Con lugar,False,PETICION,...,0,1,"[Hugo Rodríguez Mena, Jefe Regional de la Poli...",[Ministerio de Obras Públicas y Transportes],None,None,None,7302,1403,[1970-01-11 00:00:00]
1,790BE.html,*060062180007CO* Exp: 06-006218-0007-CO Res. N...,co exp co res sal constitucional cort suprem j...,94852,2006-08-22 15:23:00,Vargas Benavides,Recurso de Amparo,Con lugar parcial,False,PETICION,...,0,0,[Rodrigo Emilio Mora Arguedas],None,"[ALUDEL, RESPONSABILIDAD LIMITADA]",None,None,19096,1206,None
2,7A1BB.html,*050112300007CO* Exp: 05-011230-0007-CO Res. N...,co exp co res sal constitucional cort suprem j...,94861,NaT,Rodríguez Arroyo,Recurso de Amparo,Con lugar parcial,False,PETICION,...,0,1,"[Ana Cecilia Víquez Pérez, Rónald Traña Calvo,...","[Directora del Área de Salud de Santa Ana, Alc...",None,None,None,21974,1902,[1970-01-11 00:00:00]
3,72091.html,*050142680007CO* Exp: 05-014268-0007-CO Res: 2...,co exp co res sal constitucional cort suprem j...,94864,2005-11-30 15:35:00,Jinesta Lobo,Recurso de Amparo,Con lugar,False,TRABAJO,...,0,1,[Fernando Trejos Ballestero],None,"[Ministro de Trabajo y Seguridad, CINCO DÍAS, ...",None,None,15130,1345,[1970-01-06 00:00:00]
4,77DF3.html,*050079630007CO* Exp: 05-007963-0007-CO Res: 2...,co exp co res sal constitucional cort suprem j...,94865,2006-05-16 16:16:00,Jinesta Lobo,Recurso de Amparo,Con lugar,False,TRABAJO,...,0,0,"[Alberto Sáenz Pacheco, Presidente Ejecutivo, ...","[Directora, Ministerio Público, Sala]",[Dirección de Desarrollo de Servicios de Salud...,None,None,42474,2097,None
5,79057.html,*060091660007CO* Exp: 06-009166-0007-CO Res. N...,co exp co res sal constitucional cort suprem j...,94872,2006-08-15 08:45:00,Rodríguez Arroyo,Recurso de Amparo,Con lugar,False,PETICION,...,0,1,"[Omar Valverde Chavarría, Zeidy María Palma Gr...","[Asesor Supervisor del Circuito 08 de la, Mini...",[Dirección Regional de Educación de Pérez Zele...,None,None,17038,1586,[1970-01-06 00:00:00]
6,7B05B.html,*060107830007CO* Exp: 06-010783-0007-CO Res. N...,co exp co res sal constitucional cort suprem j...,94873,2006-10-31 17:27:00,Armijo Sancho,Recurso de Amparo,Con lugar,False,TRABAJO,...,0,1,"[Deily Juárez Castillo, Jefa del Área de Servi...",[Ministerio de Educación Pública],"[División Jurídica, Ley de la Jurisdicción Con...",None,None,6329,1439,None
7,7219C.html,*050126510007CO* Exp: 05-012651-0007-CO Res. N...,co exp co res sal constitucional cort suprem j...,94874,2006-01-27 12:43:00,Molina Quesada,Recurso de Amparo,Con lugar,False,TRABAJO,...,0,1,[William Cordero Gamboa],None,[Director General de Personal del Ministerio d...,None,None,6465,1316,[1970-01-16 00:00:00]
8,765F8.html,*050036960007CO* Exp: 05-003696-0007-CO Res. N...,co exp co res sal constitucional cort suprem j...,94881,2006-03-24 09:00:00,Vargas Benavides,Recurso de Amparo,Sin lugar,False,PRONTA RESOLUCION,...,0,1,None,None,None,None,None,8291,2288,None
9,77FA2.html,*060040610007CO* Exp: 06-004061-0007-CO Res. N...,co exp co res sal constitucional cort suprem j...,94886,2006-05-31 16:00:00,Jinesta Lobo,Recurso de Amparo,Con lugar,False,TRABAJO,...,0,0,"[Leonardo Garnier Rímolo, Mireya Wong Soto]",[Ministro de Educación Pública],[Jurisdicción Constitucional],None,None,9044,1311,None


# Cargar procesamiento de sentencias estructurales.

In [7]:
terminos = 
    'seguimiento': [r'\bseguimiento\b'],
    'se ordena': [r'\bse ordena\b'],
    'plan': [r'\bplan\b'],
    'plazo': [r'\bplazo\b']
}
seleccion = Seleccion(terminos, spark, parquet_path='./dataset', datasets_path='./datasets/estructurales')
seleccion.cargarPreprocesados()

estructurales = SentenciasEstructurales(seleccion)

AnalysisException: Path does not exist: file:/home/jovyan/Work/ej/paquetes/nlppen/dataset

# Zona de trabajo (ignorar)

In [ ]:
s = estructurales.seleccion.sdf
#print(solo_portanto(s.where(s.plazosDefinidos.isNotNull()).limit(15).toPandas().loc[1,'txt']))

In [ ]:
#print(s.where(s.plazosDefinidos.isNotNull()).limit(15).toPandas().loc[1,'se_ordena_PER'])

In [ ]:
#print(s.where(s.plazosDefinidos.isNotNull()).limit(15).toPandas().loc[1,'plazosDefinidos'])

In [ ]:
#print(s.where(s.plazosDefinidos.isNotNull()).limit(15).toPandas().loc[1])